In [1]:
import torch
import torch.nn as nn
from torchmetrics.classification import BinaryJaccardIndex, JaccardIndex

In [2]:
X = torch.rand(size=(1,3,3,8))
Y = (torch.rand(size=(1,3,3,8)) > 0.5) * 1

In [3]:
print(X)
print(Y)

tensor([[[[0.4987, 0.9323, 0.7731, 0.4984, 0.0042, 0.4259, 0.1085, 0.7959],
          [0.8765, 0.2354, 0.7015, 0.1880, 0.5579, 0.9991, 0.6100, 0.9074],
          [0.7649, 0.6507, 0.3168, 0.7895, 0.2971, 0.6455, 0.2844, 0.7239]],

         [[0.0801, 0.3525, 0.4537, 0.6679, 0.5034, 0.8054, 0.8161, 0.8696],
          [0.6863, 0.4853, 0.5560, 0.3642, 0.2946, 0.2608, 0.1858, 0.5349],
          [0.8442, 0.2089, 0.0910, 0.1345, 0.6584, 0.5410, 0.8241, 0.6125]],

         [[0.7825, 0.7658, 0.7320, 0.6494, 0.7516, 0.2208, 0.8461, 0.9959],
          [0.7288, 0.7085, 0.3228, 0.6387, 0.7044, 0.8065, 0.2559, 0.4043],
          [0.6482, 0.7820, 0.5129, 0.9962, 0.4956, 0.2468, 0.8017, 0.2022]]]])
tensor([[[[0, 0, 1, 1, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0],
          [0, 1, 1, 1, 1, 0, 0, 0]],

         [[0, 0, 0, 1, 0, 0, 1, 1],
          [1, 1, 1, 1, 0, 1, 1, 0],
          [1, 0, 0, 1, 0, 1, 0, 0]],

         [[0, 0, 1, 0, 0, 1, 0, 0],
          [0, 0, 0, 0, 1, 1, 0, 1],
          [1, 0, 

In [4]:
m1 = BinaryJaccardIndex()
m2 = JaccardIndex(task='binary', average='micro')

In [5]:
print(m1(X,Y), m2(X,Y))

tensor(0.2857) tensor(0.2857)


In [6]:
s = 0
for i in range(7):
    t = m1(X[i], Y[i])
    s += t
    print(t)
print(s, s/8)

tensor(0.2857)


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [7]:
p1 = torch.mean(X, axis = (1,2))
print(p1)

tensor([[0.6567, 0.5691, 0.4955, 0.5474, 0.4741, 0.5502, 0.5258, 0.6718]])


In [8]:
print(torch.mean(X[0]))

tensor(0.5613)


In [9]:
print(torch.mean(X[1]))

IndexError: index 1 is out of bounds for dimension 0 with size 1

In [10]:
X[0]

tensor([[[0.4987, 0.9323, 0.7731, 0.4984, 0.0042, 0.4259, 0.1085, 0.7959],
         [0.8765, 0.2354, 0.7015, 0.1880, 0.5579, 0.9991, 0.6100, 0.9074],
         [0.7649, 0.6507, 0.3168, 0.7895, 0.2971, 0.6455, 0.2844, 0.7239]],

        [[0.0801, 0.3525, 0.4537, 0.6679, 0.5034, 0.8054, 0.8161, 0.8696],
         [0.6863, 0.4853, 0.5560, 0.3642, 0.2946, 0.2608, 0.1858, 0.5349],
         [0.8442, 0.2089, 0.0910, 0.1345, 0.6584, 0.5410, 0.8241, 0.6125]],

        [[0.7825, 0.7658, 0.7320, 0.6494, 0.7516, 0.2208, 0.8461, 0.9959],
         [0.7288, 0.7085, 0.3228, 0.6387, 0.7044, 0.8065, 0.2559, 0.4043],
         [0.6482, 0.7820, 0.5129, 0.9962, 0.4956, 0.2468, 0.8017, 0.2022]]])

In [11]:
rv = torch.rand(size=(2,))
print(rv)

tensor([0.3857, 0.5424])


In [12]:
loss = nn.L1Loss(reduction='sum')

In [13]:
loss(p1, rv)

/opt/homebrew/Caskroom/miniforge/base/envs/unet/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([1, 8])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (8) must match the size of tensor b (2) at non-singleton dimension 1

In [ ]:
(torch.abs(p1[0] - rv[0]) + torch.abs(p1[1] - rv[1]))

In [14]:
torch.tensor(3*[0.345])

tensor([0.3450, 0.3450, 0.3450])

In [15]:
x = torch.Tensor([1,2,2,1,3,2,3,1,1,1,2,3,2,2,3,3,3])

In [16]:
(x == 1) * 1

tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [17]:
(x == 1) * 1 + (x == 2)*1

tensor([1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0])